In [1]:
import os
import pandas as pd
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
rootpath = os.path.dirname(os.path.abspath(""))
datapath = os.path.join(rootpath, 'data')
print(f"datapath: {datapath}")

datapath: /home/onur/repos/nytaxi-spark/data


In [3]:
spark = SparkSession.builder \
    .master('local[2]') \
    .appName('taxi_groupby') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 13:28:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read and Combine Green&Yellow Data

In [4]:
df_green = spark.read.parquet(os.path.join(datapath, 'pq', 'green', '*', '*'))

In [7]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
df_green.registerTempTable('green')

/home/onur/opt/spark355/spark-3.5.5-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


### SQL

In [28]:
query = """
select 
-- Revenue grouping 
date_trunc("hour", lpep_pickup_datetime) as hour,
PULocationID as rzone,
-- Revenue calculation 
sum(total_amount) as amount,
COUNT(1) as number_records
from green
WHERE
lpep_pickup_datetime >= '2020-01-01'
group by 
1,2
order by
1,2
"""
df_green_hr = spark.sql(query)
df_green_hr.show()

+-------------------+-----+------------------+--------------+
|               hour|rzone|            amount|number_records|
+-------------------+-----+------------------+--------------+
|2020-01-01 00:00:00|    7| 769.7299999999996|            45|
|2020-01-01 00:00:00|   17|195.03000000000003|             9|
|2020-01-01 00:00:00|   18|               7.8|             1|
|2020-01-01 00:00:00|   22|              15.8|             1|
|2020-01-01 00:00:00|   24|              87.6|             3|
|2020-01-01 00:00:00|   25| 531.0000000000002|            26|
|2020-01-01 00:00:00|   29|              61.3|             1|
|2020-01-01 00:00:00|   32| 68.94999999999999|             2|
|2020-01-01 00:00:00|   33|317.27000000000004|            11|
|2020-01-01 00:00:00|   35|            129.96|             5|
|2020-01-01 00:00:00|   36|295.34000000000003|            11|
|2020-01-01 00:00:00|   37|            175.67|             6|
|2020-01-01 00:00:00|   38| 98.78999999999999|             2|
|2020-01

In [24]:
df_green.write.parquet(os.path.join(datapath, 'report', 'revenue_hourly_green'))

In [29]:
df_yellow = spark.read.parquet(os.path.join(datapath, 'pq', 'yellow', '*', '*'))
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [31]:
df_yellow.registerTempTable('yellow')

In [32]:
query = """
select 
-- Revenue grouping 
date_trunc("hour", tpep_pickup_datetime) as hour,
PULocationID as rzone,
-- Revenue calculation 
sum(total_amount) as amount,
COUNT(1) as number_records
from yellow
WHERE
tpep_pickup_datetime >= '2020-01-01'
group by 
1,2
order by
1,2
"""
df_yellow_hr = spark.sql(query)
df_yellow_hr.show()

+-------------------+-----+------------------+--------------+
|               hour|rzone|            amount|number_records|
+-------------------+-----+------------------+--------------+
|2020-01-01 00:00:00|    3|              25.0|             1|
|2020-01-01 00:00:00|    4|1004.3000000000002|            57|
|2020-01-01 00:00:00|    7| 455.1700000000001|            38|
|2020-01-01 00:00:00|   10|             42.41|             2|
|2020-01-01 00:00:00|   12|             107.0|             6|
|2020-01-01 00:00:00|   13|1214.8000000000002|            56|
|2020-01-01 00:00:00|   14|               8.8|             1|
|2020-01-01 00:00:00|   15|             34.09|             1|
|2020-01-01 00:00:00|   17|220.20999999999998|             8|
|2020-01-01 00:00:00|   18|               5.8|             1|
|2020-01-01 00:00:00|   24| 754.9500000000002|            45|
|2020-01-01 00:00:00|   25|            324.35|            16|
|2020-01-01 00:00:00|   32|              18.0|             1|
|2020-01

In [35]:
df_yellow.write.parquet(os.path.join(datapath, 'report', 'revenue_hourly_yellow'), mode='overwrite')

In [36]:
spark.sparkContext.stop()